## Эмулятор машины Тьюринга

Описание машины Тьюринга должно быть в файле, формат следующий:

<текущее состояние> <текущий символ> <новый символ> <направление> <новое состояние>

<текущее состояние> <текущий символ> <новый символ> <направление> <новое состояние>

...

Обозначения:

Начальное состояние должно называться "0"

Конечное состояние "halt" (или "halt-accept", "halt-reject")

Символы могут быть любыми, но "_" - это пустой символ, "*" - это обозначение любого символа

Направление: "l" = влево, "r" = вправо, "*" = стоять на месте

In [1]:
# for description states of Machine

class desc():
    def __init__(self, state, sym, direction):
        self.state = state
        self.symbol = sym
        self.direction = direction
        
    def __str__(self):
        return self.state + ' ' + self.symbol + ' ' + self.direction
    def __repr__(self):
        return self.state + ' ' + self.symbol + ' ' + self.direction

In [2]:
# to save and change current state of Machine

class curr():
    def __init__(self, string, position, state):
        self.string = string
        self.position = position
        self.state = state
        
    def __str__(self):
        i=0
        while i<len(self.string) and self.string[i]=='_':
            i += 1
        j=len(self.string)-1
        while j>=0 and self.string[j]=='_':
            j -= 1
        if i>=j+1:
            return '_'
        return self.string[i:(j+1)]
    
    @property
    def symbol(self):
        return self.string[self.position]
    
    def change_string(self, next_sym, new_position):
        pos = self.position
        if new_position==-1:
            self.string = '_' + next_sym + self.string[(pos+1):]
            self.position = 0
        elif new_position==len(self.string):
            self.string = self.string[:pos] + next_sym + '_'
            self.position = new_position
        else:
            self.string = self.string[:pos] + next_sym + self.string[(pos+1):]
            self.position = new_position

    @property
    def check_state(self):
        if self.state=='halt' or self.state=='halt-accept' or self.state=='halt-reject':
            return 0
        return 1

In [3]:
# realization

class Turing_Machine():
    def __init__(self, path, string = '_'):
        description = open(path)
        self.states = {(line.split()[0], line.split()[1]): desc(line.split()[4], line.split()[2], line.split()[3]) for line in description}
        description.close()
        self.current = curr(string, 0, '0')
        
    def do_step(self):
        sym = self.current.symbol
        
        new = self.states.get((self.current.state, sym))
        if new==None:
            new = self.states.get((self.current.state, '*'))
        
        self.current.state = new.state
        
        direct = new.direction
        next_sym = new.symbol if new.symbol!='*' else sym
        
        new_position = self.current.position
        if direct=='l':
            new_position -= 1
        elif direct=='r':
            new_position += 1
        
        self.current.change_string(next_sym, new_position)
        
    def execution(self):
        while self.current.check_state:
            self.do_step()
        print(self.current)
    
    def execution_print(self):
        while self.current.check_state:
            self.do_step()
            print(self.current)
        print 'The End!'
    
    def set_string(self, string, pos = 0):
        self.current.string = string
        self.current.state = '0'
        self.current.position = pos
    
    def one_step(self):
        if self.current.check_state:
            self.do_step()
        print(self.current)
        if self.current.check_state==0:
            print 'The End!'
            return 0
        return 1

In [4]:
path1 = 'palindrome.txt'
path2 = 'division.txt'

string1 = '1001001001'
string2 = '1001011'
string3 = '110011'

string4 = '11111011'
string5 = '1101'

Создаем 2 машины:

In [5]:
check_palindrome = Turing_Machine(path1, string1)
# Проверка числа из 0 и 1 на палиндром

unary_division = Turing_Machine(path2, string4)
# Унарное деление, 0 - разделитель между делимым и делителем, в ответе 0 разделитель между частным и остатком

Обычный запуск:

In [6]:
check_palindrome.execution()

:)


In [7]:
unary_division.execution()

1101


Меняем начальную строку:

In [8]:
check_palindrome.set_string(string2)
unary_division.set_string(string5)

Печатаем все промежуточные шаги:

In [9]:
check_palindrome.execution_print()

001011
001011
001011
001011
001011
001011
001011
001011
00101
00101
00101
00101
00101
00101
00101
0101
0101
0101
0101
0101
0101
0101
010
01
0
_
:
:(
The End!


In [10]:
unary_division.execution_print()

1101
1101
1101
1101
110:
110:
1:0:
1:0:
1:0:
1:0:
1:0:_1
1:0:_1
1:01_1
1:01_1
1%01_1
1%01_1
1%01_1
1%01_1
1%0:_1
1%0:_1
1%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_11
:%0:_11
:%0:_11
:%01_11
:%01_11
:%01_11
%%01_11
%%01_11
%01_11
01_11
1_11
11
11
The End!


Для выполнения по шагам:

In [11]:
check_palindrome.set_string(string3)

In [12]:
check_palindrome.one_step()

10011


1

In [13]:
check_palindrome.one_step()

10011


1

In [14]:
while check_palindrome.one_step():
    pass

10011
10011
10011
10011
10011
1001
1001
1001
1001
1001
1001
001
001
001
001
001
00
00
00
00
0
0
0
_
_
:
:)
The End!
